In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split, KFold
import random

from sklearn.model_selection import cross_validate
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing import text, sequence
from keras.layers import Embedding, SpatialDropout1D
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input
from keras.optimizers import RMSprop
import keras.backend as K
from keras.layers import Dense, Input, GRU, LSTM, Bidirectional, Dropout, CuDNNLSTM, CuDNNGRU
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
# GOOGLE COLAB SETUP

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#2. Get the file
data_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/data'
codes_path    = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/codes'
cv_models_path     = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/cv_models'
models_path        = 'drive/My Drive/Colab Notebooks/adaptHAN/AOBDL_code/models'


#3. Read file as panda dataframe
train         = pd.read_csv(f'{data_path}/train_cleaned_no_punkt.csv') 
test_labelled = pd.read_csv(f'{data_path}/test_labelled_cleaned_no_punkt.csv') 
test_unlabelled = pd.read_csv(f'{data_path}/test_unlabelled_cleaned_no_punkt.csv') 

In [0]:
train['mal'] = train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
train.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
train.comment_text.fillna("empty", inplace=True)

test_labelled['mal'] = test_labelled[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].sum(axis=1) >= 1  
test_labelled.drop(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], axis=1, inplace=True)
test_labelled.comment_text.fillna("empty", inplace=True)

test_unlabelled.comment_text.fillna("empty", inplace=True)

# CHANGE TRAIN AND TEST, MIX TO GET SIMILAR DISTRIBUTION
from sklearn.model_selection import train_test_split
rs=42
X_train1, X_test1, y_train1, y_test1  = train_test_split(train.drop('mal', axis=1), train.mal, stratify=train.mal, test_size=0.29, random_state=rs )
X_train2, X_test2, y_train2, y_test2  = train_test_split(test_labelled.drop('mal', axis=1), test_labelled.mal, stratify=test_labelled.mal, test_size=0.29, random_state=rs)

X = np.concatenate((X_train1.comment_text, X_train2.comment_text))
y = np.concatenate((y_train1, y_train2))

X_test = np.concatenate((X_test1.comment_text, X_test2.comment_text))
y_test = np.concatenate((y_test1, y_test2))

In [0]:
#X = train.comment_text
#y = train.mal

max_features = 40000
maxlen       = 400
dropout_rate = 0
rs           = 42
epochs       = 4
batch_size   = 250
embed_dim    = 50
rec_units    = 150

In [0]:
def gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units):
    if K.backend == 'tensorflow':        
        K.clear_session()
    input_layer = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features, output_dim=embed_dim, trainable=True)(input_layer)
    x = SpatialDropout1D(dropout_rate)(embedding_layer)
    x = CuDNNLSTM(units=rec_units, return_sequences=False)(x)
    output_layer = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy',
                  optimizer=RMSprop(clipvalue=1, clipnorm=1),
                  metrics=['acc'])
    #print( model.summary())
    return model

In [0]:
kf = StratifiedKFold(n_splits=5, random_state=rs)
auc = []
roc = []
c = 0
#tokenizer = text.Tokenizer(num_words=max_features)
#tokenizer.fit_on_texts(X)

In [8]:
for train_index, val_index in kf.split(X, y):
    print(f' fold {c}')
    X_train, X_val       = X[train_index], X[val_index]
    y_train, y_val       = y[train_index], y[val_index] 
    
    # Tokenization within the CV to ensure not having overfitting
    tokenizer            = text.Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(X_train)
    
    list_tokenized_train = tokenizer.texts_to_sequences(X_train)
    list_tokenized_val   = tokenizer.texts_to_sequences(X_val)
    X_train              = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
    X_val                = sequence.pad_sequences(list_tokenized_val, maxlen=maxlen)
    model                = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)
    print('Fitting')
    history              = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_val, y_val), epochs=4, shuffle=True, verbose=1)
    model.save_weights(f'{cv_models_path}/LSTM_fold_{c}.h5')
    probs                = model.predict(X_val, batch_size=batch_size, verbose=1)
    auc_f                = average_precision_score(y_val, probs)
    auc.append(auc_f)
    roc_f                = roc_auc_score(y_val, probs)
    roc.append(roc_f)
    print(f' average precision {auc_f}')
    print(f' roc auc {roc_f}')
    
    
    c += 1
    del model
    K.clear_session()

 fold 0


W0714 21:57:39.130439 140185926367104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0714 21:57:39.145073 140185926367104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0714 21:57:39.148287 140185926367104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0714 21:57:39.756101 140185926367104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0714 21:57:39.763759 140185926367104 deprecation_wrappe

Fitting


W0714 21:57:40.172237 140185926367104 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 34s 265us/step - loss: 0.1855 - acc: 0.9315 - val_loss: 0.1266 - val_acc: 0.9529
Epoch 2/4
126974/126974 [==============================] - 32s 250us/step - loss: 0.1246 - acc: 0.9522 - val_loss: 0.1108 - val_acc: 0.9591
Epoch 3/4
126974/126974 [==============================] - 32s 249us/step - loss: 0.1118 - acc: 0.9563 - val_loss: 0.1109 - val_acc: 0.9599
Epoch 4/4
31745/31745 [==============================] - 2s 75us/step
 average precision 0.8858410075904546
 roc auc 0.9755514503772089
 fold 1
Fitting
Train on 126974 samples, validate on 31745 samples
Epoch 1/4
126974/126974 [==============================] - 32s 253us/step - loss: 0.1889 - acc: 0.9325 - val_loss: 0.1168 - val_acc: 0.9584
Epoch 2/4
126974/126974 [==============================] - 32s 250us/step - loss: 0.1201 - acc: 0.9532 - val_loss: 0.1147 - val_acc: 0.9590
Epoch 3/4
126974/126974 [=====================

In [0]:
data = pd.DataFrame({'acc':history.history['acc'],
                    'loss': history.history['loss'],
                    'val_acc': history.history['val_acc'],
                    'val_loss': history.history['val_loss']})

In [10]:
data

,acc,loss,val_acc,val_loss
0,0.933287,0.187099,0.920896,0.183150
1,0.959331,0.110413,0.918565,0.172835
2,0.962426,0.100081,0.935009,0.148495
3,0.963702,0.095075,0.937561,0.148154


In [11]:
np.array(auc).mean()

0.859716281364518

In [12]:
np.array(roc).mean()

0.9722413063523423

In [13]:
X_train = X
y_train = y

tokenizer            = text.Tokenizer(num_words=max_features, oov_token='unknown')
tokenizer.fit_on_texts(X_train)

list_tokenized_train = tokenizer.texts_to_sequences(X_train)
list_tokenized_test  = tokenizer.texts_to_sequences(X_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

model = gru_keras(max_features, maxlen, dropout_rate, embed_dim, rec_units)

y_train              = np.array(y_train)
y_test               = np.array(y_test)

print('Fitting')
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, shuffle=True, verbose=1)
probs = model.predict(X_test, batch_size=batch_size, verbose=1)
auc_f = average_precision_score(y_test, probs)
roc_f = roc_auc_score(y_test, probs)
model.save_weights(f'{models_path}/LSTM.h5')

Fitting
Epoch 1/4
158719/158719 [==============================] - 37s 235us/step - loss: 0.1791 - acc: 0.9354
Epoch 2/4
158719/158719 [==============================] - 37s 230us/step - loss: 0.1163 - acc: 0.9554
Epoch 3/4
158719/158719 [==============================] - 37s 230us/step - loss: 0.1080 - acc: 0.9578
Epoch 4/4
64830/64830 [==============================] - 5s 74us/step


In [14]:
auc_f

0.8545288859087152

In [15]:
roc_f

0.9717264962301738